In [81]:
#George Christodoulou
#Aristidis Chronarakis

In [82]:
#Libraries
import networkx as nx
import matplotlib.pyplot as plt

In [83]:
#The loadingSN function loads for each User his skills and the relations with other Users from files in order to constut the Social Network.   
def loadingSN():
    skillsMapping={}
    availableSkills=['Acceleration','Aggression','Agility','Balance','Ball control','Composure','Crossing','Curve','Dribbling','Finishing','Free kick accuracy','GK diving','GK handling','GK kicking','GK positioning','GK reflexes','Heading accuracy','ID','Interceptions','Jumping','Long passing','Long shots','Marking','Penalties','Positioning','Reactions','Short passing','Shot power','Sliding tackle','Sprint speed','Stamina','Standing tackle','Strength','Vision','Volleys']
    with open ("PlayerAttributeData.csv") as fopen:
        fopen.next()
        for line in fopen:
            splitted=line.split(",")
            skillsList=[]
            for i in range(1,len(splitted)):
                if int(splitted[i]) > 74:
                    skillsList.append(availableSkills[i-1])
            skillsMapping.setdefault(splitted[0],skillsList)
    inUsers = {}
    outUsers = {}
    skills = {}
    with open("soc-sign-bitcoinalpha.csv") as fopen:
            for line in fopen:
                splitted = []
                splitted = line.split(",")
                if (splitted[2].startswith("-")):
                    userB = splitted[0]
                    userA = splitted[1]
                else:
                    userA = splitted[0]
                    userB = splitted[1]
                inUsers.setdefault(userA,[]).append(userB)
                inUsers.setdefault(userB,[])
                outUsers.setdefault(userB,[]).append(userA)
                outUsers.setdefault(userA,[])
                skills.setdefault(userA,skillsMapping[userA])
                skills.setdefault(userB,skillsMapping[userB])
    
    socialNetwork={}
    for user in inUsers.keys():
        socialNetwork.setdefault(user,[]).append(inUsers[user])
        socialNetwork.setdefault(user,[]).append(outUsers[user])
        socialNetwork.setdefault(user,[]).append(skills[user])
    return socialNetwork


In [84]:
#The preProcessingSN function is used for removing Users and their relations if they don't have at least one skill from skillset.This function is also used for finding the Users with the rarest skill from skillset.
def preProcessingSN(socialNetwork,skillset):
    skillsOccurrences={}
    deletedUsers=[]
    for user in socialNetwork.keys():
        if bool(set(skillset).intersection(socialNetwork[user][2])):
            for skill in socialNetwork[user][2]:
                if skill in skillset:
                    skillsOccurrences[skill]=skillsOccurrences.setdefault(skill,0)+1
        else:
            del socialNetwork[user]
            deletedUsers.append(user)
    for user in socialNetwork.keys():
        socialNetwork[user][0]=list(set(socialNetwork[user][0]).difference(deletedUsers))
        socialNetwork[user][1]=list(set(socialNetwork[user][1]).difference(deletedUsers))
    rarestSkill=sorted(skillsOccurrences)[0]
    usersWithRarestSkill=[]
    for user in socialNetwork.keys():
        if rarestSkill in socialNetwork[user][2]:
            usersWithRarestSkill.append(user)
    return  socialNetwork,usersWithRarestSkill

In [85]:
#The hasFullSkillset function is used for checking if the needed skillset has been fulfilled.
def hasFullSkillset(socialNetworkProcessed,team,skillset):   
    tempSkills=[]
    for user in team:
        for skill in socialNetworkProcessed[user][2]:
            if skill not in tempSkills:
                tempSkills.append(skill)
    if set(skillset) < set(tempSkills):
        return True
    else:
        return False

In [86]:
#The userCreateCycles function is used for checking if the addition of User in Team create Cycles in Team Network.  
def userCreateCycles(socialNetworkProcessed,team,user):
    testTeam=team
    testTeam.append(user)
    testGraph=nx.DiGraph()
    for user in testTeam:
        for inUser in socialNetworkProcessed[user][0]:
            testGraph.add_edge(inUser,user)
        for outUser in socialNetworkProcessed[user][1]:
            testGraph.add_edge(user,outUser)
    try:
        tempList =list(nx.find_cycle(testGraph, orientation='original'))
        print "We will have a status cycle if we add user to team"
        return True
    except:
        return False

In [87]:
#The directedSNtoUndirected function is used for modifying the directed approach of Social Network in which a User has in-coming and out-coming neighbours to an undirected approach about his neighbours and which is used in our Social Network traversal.  
def directedSNtoUndirected(socialNetworkProcessed):
    socialNetworkUndirected={}
    for key in socialNetworkProcessed.keys():
        socialNetworkUndirected[key]=list((socialNetworkProcessed[key][0])+(socialNetworkProcessed[key][1]))
    return socialNetworkUndirected  

In [88]:
#The removeNeedlessUsers function is used for removing Users from Social Network which has skills that are also fulfilled from other Users and their removal don't affect the Social Network's connected component. 
def removeNeedlessUsers(socialNetworkProcessed,team,skillset):
    temporaryGraph=nx.DiGraph()
    for member in team:
        temporaryGraph.add_node(member)
        for user in socialNetworkProcessed[member][1]:
            if user in team:
                temporaryGraph.add_edge(member,user)
    teamGraph=temporaryGraph.copy()
    finalGraph = temporaryGraph.to_undirected()
    for member in teamGraph.nodes():
        if(finalGraph.number_of_nodes()>1):
            temp=finalGraph.copy()
            temp.remove_node(member)
            team.remove(member)
            if(nx.is_connected(temp)==True and hasFullSkillset(socialNetworkProcessed,team,skillset)==True):
                finalGraph.remove_node(member)
            else:
                team.append(member)
    return list(finalGraph)

In [89]:
##The TFWS is the function which implements the team formation with status algorithm.(More Details on Report)
def TFWS(socialNetwork,skillset):
    socialNetworkProcessed,usersWithRarestSkill=preProcessingSN(socialNetwork,skillset)
    socialNetworkUndirected=directedSNtoUndirected(socialNetworkProcessed)
    for startingUser in usersWithRarestSkill:
        team=[]
        explored = []
        queue = [startingUser]
        while queue :
            user = queue.pop(0)
            if user not in explored:
                if userCreateCycles(socialNetworkProcessed,team,user)==False:
                    team.append(user)
                explored.append(user)
                neighbours = socialNetworkUndirected[user]
                for neighbour in neighbours:
                    queue.append(neighbour)
            if hasFullSkillset(socialNetworkProcessed,team,skillset)==True:
                print team
                print "We find a team which has a respect on status theory and fulfills the needed skillset"
                team=removeNeedlessUsers(socialNetworkProcessed,team,skillset)
                return team
    print "Unable to find the desired team"
    return false

In [90]:
if __name__ == '__main__':
    socialNetwork=loadingSN()
    skillset=['Dribbling','Agility','Balance','Reactions']
    team=TFWS(socialNetwork,skillset)

We will have a status cycle if we add user to team
['592']
We find a team which has a respect on status theory and fulfills the needed skillset
